## The `CompsynConfig` class

`compsyn.utils.CompsynConfig` provides a convenient way to setup your runtime configuration through code.

```
class CompsynConfig:
    def __init__(self, **kwargs: Dict[str, str]) -> None:
        self.config = dict()
        for key, val in kwargs.items():
            set_env_var(key, val)  # sets passed config value in os.environ
            self.config[key] = val # record on config object for convenience
```

It is possible to configure compsyn entirely using environment variables, but this class provides a more code-friendly way for setting relevant environment variables.

In [2]:
from compsyn.config import CompsynConfig

print(CompsynConfig())

CompsynConfig
	experiment_name      = default-experiment
	trial_id             = default-trial
	hostname             = default-hostname
	trial_timestamp      = 2021-03-21
	work_dir             = None
	jzazbz_array         = jzazbz_array.npy
	s3_bucket            = None
	s3_region_name       = None
	s3_endpoint_url      = None
	s3_access_key_id     = None
	s3_secret_access_key = None


The `CompsynConfig` class sets these values in `os.environ`, so that other parts of the code can access them.

In [7]:
from compsyn.trial import get_trial

config = CompsynConfig(experiment_name="regional-search-patterns", trial_id="phase-0", hostname="shared-notebook-user")

trial = get_trial()

[2021-03-21 13:06:51,746] (compsyn.set_env_var)  WARNING: existing environment value (default-experiment) for COMPSYN_EXPERIMENT_NAME will be overwritten with regional-search-patterns
[2021-03-21 13:06:51,747] (compsyn.set_env_var)  WARNING: existing environment value (default-trial) for COMPSYN_TRIAL_ID will be overwritten with phase-0
[2021-03-21 13:06:51,748] (compsyn.set_env_var)  WARNING: existing environment value (default-hostname) for COMPSYN_HOSTNAME will be overwritten with shared-notebook-user
[2021-03-21 13:06:51,751] (compsyn.Trial)  WARNING: no work_dir passed, using a temporary directory: /var/folders/4m/lkjsk_gs4pb08h8hnyq399380000gn/T/tmp2tvwf9hv
[2021-03-21 13:06:51,752] (compsyn.Trial)  INFO: work_dir: /var/folders/4m/lkjsk_gs4pb08h8hnyq399380000gn/T/tmp2tvwf9hv
[2021-03-21 13:06:51,752] (compsyn.Trial)  INFO: experiment: regional-search-patterns
[2021-03-21 13:06:51,753] (compsyn.Trial)  INFO: trial_id: phase-0
[2021-03-21 13:06:51,754] (compsyn.Trial)  INFO: hostna

In [8]:
import os; print(os.getenv("COMPSYN_EXPERIMENT_NAME"))

regional-search-patterns


Note that it is not required to use this class if variables are set in the environment already (or even without doing th at, if the default values will suffice).

In [9]:
for key, val in os.environ.items():
    if key.startswith("COMPSYN_"):
        del os.environ[key] # simulate an unset environment
        
del config
del trial

trial = get_trial()
print(trial)
print()
print(CompsynConfig())

[2021-03-21 13:07:04,307] (compsyn.Trial)  WARNING: no work_dir passed, using a temporary directory: /var/folders/4m/lkjsk_gs4pb08h8hnyq399380000gn/T/tmp98e9i9k4
[2021-03-21 13:07:04,308] (compsyn.Trial)  INFO: work_dir: /var/folders/4m/lkjsk_gs4pb08h8hnyq399380000gn/T/tmp98e9i9k4
[2021-03-21 13:07:04,308] (compsyn.Trial)  INFO: experiment: default-experiment
[2021-03-21 13:07:04,309] (compsyn.Trial)  INFO: trial_id: default-trial
[2021-03-21 13:07:04,310] (compsyn.Trial)  INFO: hostname: default-hostname
Trial
	experiment_name = default-experiment
	trial_id        = default-trial
	hostname        = default-hostname

CompsynConfig
	experiment_name      = default-experiment
	trial_id             = default-trial
	hostname             = default-hostname
	trial_timestamp      = 2021-03-21
	work_dir             = None
	jzazbz_array         = jzazbz_array.npy
	s3_bucket            = None
	s3_region_name       = None
	s3_endpoint_url      = None
	s3_access_key_id     = None
	s3_secret_access_

In [ ]:
CompSynConfig(
    experiment_name="regional-differences",
    trial_id="phase-0",
    hostname="notebook-toronto",
    "")